In [1]:
import os, random
import numpy as np
import pandas as pd
from ase.io import read
from collections import Counter

In [2]:
train = read('./data/train.xyz', format='extxyz', index=':') # 전체 데이터 불러오기
test = read('./data/test.xyz', format='extxyz', index=':')
sample = pd.read_csv('./data/sample_submission.csv')

In [3]:
pbc_false = []
for i in range(len(train)):
    if all(train[i].get_pbc()) == True:
        pbc_false.append(i)
len(pbc_false)

22510

In [11]:
train[0].get_all_distances()

array([[0.        , 5.51282157, 8.22710674, ..., 1.91129889, 7.28735364,
        6.98110776],
       [5.51282157, 0.        , 6.47108804, ..., 6.57678419, 4.90324216,
        5.14890418],
       [8.22710674, 6.47108804, 0.        , ..., 9.8611769 , 1.99885371,
        6.3282198 ],
       ...,
       [1.91129889, 6.57678419, 9.8611769 , ..., 0.        , 8.74059431,
        7.50079779],
       [7.28735364, 4.90324216, 1.99885371, ..., 8.74059431, 0.        ,
        4.47913293],
       [6.98110776, 5.14890418, 6.3282198 , ..., 7.50079779, 4.47913293,
        0.        ]])

In [15]:
train[0].get_distances(0,list(range(0,len(train[0]))))

array([0.        , 5.51282157, 8.22710674, 7.32245684, 7.02340184,
       3.37763346, 4.09607833, 7.27535402, 4.85300843, 9.3056394 ,
       8.81296718, 5.32833489, 8.28844131, 6.40193451, 5.90135362,
       5.61767067, 3.25037304, 5.67908692, 8.74022573, 7.85118245,
       4.92536973, 6.0365701 , 5.02294508, 2.40169829, 8.08673873,
       3.42860131, 7.9395052 , 4.87020267, 3.89036756, 5.39307476,
       8.4446799 , 4.97513487, 6.71636515, 6.57293367, 6.54125727,
       5.4847141 , 4.12194436, 2.23468234, 6.97530837, 6.05238565,
       1.82570384, 8.79845049, 5.24476827, 6.3679875 , 7.6258493 ,
       1.91129889, 7.28735364, 6.98110776])

In [13]:
sequence_train, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], []

symbols = []
volumes = []



for i in range(len(train)):
    mole = train[i] # 각 분자 - 22510개

    atoms = len(mole) # 원자 개수 - 48개
    sequence_train.append(atoms) # 22510개의 mole에 들어있는 atoms의 개수를 저장

    position = mole.get_positions() # 원자 위치 정보 -> (48, 3)
    force = mole.get_forces() # label 1 -> (48, 3)
    symbol = mole.get_chemical_symbols()

    energy = mole.get_total_energy() # label 2 -> float 값 하나
    energies.append(energy)

    for j in range(len(mole)): # 각 원자에 대해 반복 -> 48회
        atom = mole[j] # i번 분자의 j번째 원자

        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        symbols.append(symbol[j])
        forces.append(force[j])

train_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'symbol':symbols, 'force':forces}) # sequence_train, symbols, energies 아직 사용 안 함
train_df.head()

,position_x,position_y,position_z,symbol,force
0,1.591737,4.200483,7.832245,N,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,N,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,N,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,N,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,N,"[-4.65938123, -0.77685475, -3.07403915]"


In [19]:
train_df

,position_x,position_y,position_z,symbol,force
0,1.591737,4.200483,7.832245,N,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,N,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,N,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,N,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,N,"[-4.65938123, -0.77685475, -3.07403915]"
...,...,...,...,...,...
1284970,9.495734,3.501358,5.231880,N,"[2.449508, 21.06236726, 48.41937431]"
1284971,0.715535,1.041657,9.207530,N,"[-0.21919686, 1.74001551, 0.66362062]"
1284972,7.630116,3.670234,7.966260,N,"[0.79347178, -2.8472516, -0.91132274]"
1284973,0.177211,8.660930,7.082750,N,"[-0.20181181, -0.0042076, -0.14967853]"
